In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import sidetable as stb

pd.options.display.max_columns = None

In [2]:
df = pd.read_csv("../data/03-tiburon_4.csv", index_col = 0)
df.reset_index(drop= True, inplace= True) # Reseteamos el número de los índices y borramos la columna index que se nos generaría
df.head(2)

,year,type,country,activity,age,species,month,fatal,gender
0,2018,Boating,usa,Paddling,57.0,White,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger,Jun,Y,M


# Pair Programming Limpieza V
En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza III donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

1. Es el momento de eliminar los nulos:
- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.
- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.
- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.
- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.
- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.
- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?


In [3]:
df.stb.missing()

,missing,total,percent
age,156,1669,9.346914
month,152,1669,9.107250
fatal,99,1669,5.931696
activity,31,1669,1.857400
gender,14,1669,0.838826
country,10,1669,0.599161
year,0,1669,0.000000
type,0,1669,0.000000
species,0,1669,0.000000


In [4]:
# Creamos una función para reemplazar los nulos con el metodo SimpleImputer
# Tendremos que pasarle el dataframe que queramos usar, la columna y la estrategia a usar para sustituir los nulos
def reemp_nulos_simpleimputer(dataf, col,strat):
    # Iniciamos el método
    imputer = SimpleImputer(strategy=strat, missing_values = np.nan)
    # Usamos la función fit para ajustar el imputer en la columna sobre la que trabajamos
    imputer = imputer.fit(dataf[[col]])
    # Usamos el transform para rellenar los valores nulos
    dataf[col] = imputer.transform(dataf[[col]])
    # Pedimos que nos devuelva la suma de valores nulos de la columna para comprobar si se han borrado
    return dataf[col].isnull().sum()

In [5]:
# Creamos una función para reemplazar los nulos con el metodo IterativeImputer y sustituirlas directamente
# Tendremos que pasarle el dataframe que queramos usar y la estrategia a usar para sustituir los nulos
def reemp_nulos_iterativeimputer(dataf, strat):
    # Primero sacamos las variables numéricas con select_dtypes
    num = dataf.select_dtypes(include = np.number)
    # Creamos una instancia del metodo con las caracteristicas que queremos para sustituir los nulos
    imputer = IterativeImputer(initial_strategy = strat, missing_values= np.nan, imputation_order= "ascending")
    # Aplicamos sobre nuestras variables numericas
    imputer.fit(num)
    # Transformamos los datos para que se transformen los nulos, y como nos devuelve un array
    # lo transformamos directamente en dataframe
    numericas_trans = pd.DataFrame(imputer.transform(num), columns = num.columns)
    # Sacamos el nombre de las columnas del data sin nulos
    columnas = numericas_trans.columns
    # Utilizando esta lista de columnas, eliminamos estas del dataframe
    dataf.drop(columnas, axis= 1, inplace = True)
    # creamos nuevas columnas en el dataframe original, basándonos en estas columnas numericas sin nulos
    dataf[columnas] = numericas_trans[columnas]
    # Nos devuelve la suma de nulos del dataframe, para comprobar que se hayan borrado los nulos
    return dataf.isnull().sum()

In [6]:
# Creamos una función para reemplazar los nulos con el metodo KNNImputer y sustituirlas directamente
# Tendremos que pasarle el dataframe que queramos usar
def reemp_nulos_num_KNN(dataf):
    # Primero sacamos las variables numéricas con select_dtypes
    num = dataf.select_dtypes(include = np.number)
    # Iniciamos el KNNimputer
    imputerKNN = KNNImputer(n_neighbors=5)
    # Aplicamos el imputer a las variables numericas
    imputerKNN.fit(num)
    # Transformamos los datos para que se transformen los nulos, y como nos devuelve un array
    # lo transformamos directamente en dataframe
    df_knn = pd.DataFrame(imputerKNN.transform(num), columns = num.columns)
    # Sacamos el nombre de las columnas del data sin nulos
    columnas = df_knn.columns
    # Utilizando esta lista de columnas, eliminamos estas del dataframe
    dataf.drop(columnas, axis= 1, inplace = True)
    # creamos nuevas columnas en el dataframe original, basándonos en estas columnas numericas sin nulos
    dataf[columnas] = df_knn[columnas]
    # Nos devuelve la suma de nulos del dataframe, para comprobar que se hayan borrado los nulos
    return dataf.isnull().sum()

1. Es el momento de eliminar los nulos:
- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [7]:
reemp_nulos_simpleimputer(df, "age", "mean")

0

- Reemplazad los valores nulos de la columna gender por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [8]:
reemp_nulos_simpleimputer(df, "gender", "most_frequent")

0

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [9]:
reemp_nulos_simpleimputer(df, "type", "most_frequent")

0

- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [10]:
# Hacemos una copia para poder usar los dos métodos
df2 = df.copy()

In [11]:
reemp_nulos_num_KNN(df2)

type          0
country      10
activity     31
species       0
month       152
fatal        99
gender        0
year          0
age           0
dtype: int64

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [12]:
# Con este método modificamos ya el original para guardarlo sin nulos
reemp_nulos_iterativeimputer(df, "median")

type          0
country      10
activity     31
species       0
month       152
fatal        99
gender        0
year          0
age           0
dtype: int64

- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

En Iterative-Imputer comprueba los nulos con otras variables y realiza una estimación. En este caso podemos escoger la estrategia para modificar los nulos, entre la media, la mediana, la moda (most_frequent) y constant(reemplaza los valores faltantes con fill_value).

Con KNN Imputer lo que hace sería cambiar el valor por los vecinos más cercanos (promedia las distancias entre los vecinos más cercanos). Se le puede indicar el número de vecinos que queremos que considere. Por defecto, le pasamos 5.

2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [13]:
df.to_csv("../data/05-tiburon_4.csv")